In [1]:
import sys
import logging
import argparse
import numpy as np

sys.path.insert(0, '../../../network')
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

In [2]:
T = 0.35

phi = ErrorFunction(mu=0.22, sigma=0.1).phi
exc = Population(N=40000, tau=1e-2, phi=phi)

P, S = 1, 16
sequences = [GaussianSequence(S,exc.size,seed=i) for i in range(P)]
patterns = np.stack([s.inputs for s in sequences])

conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005)
synapse = LinearSynapse(conn_EE.K, A=1)
conn_EE.store_sequences(patterns, synapse.h_EE)

net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

logging.info("Simulating normal retrieval")
r0 = exc.phi(patterns[0,0,:])
net.simulate(T, r0=r0)

M = np.mean(net.exc.state**2, axis=0)
overlaps = sequences[0].overlaps(net, exc)
correlations = sequences[0].overlaps(net, exc, correlation=True)
phi_patterns = phi(patterns)
state = np.copy(net.exc.state)

logging.info("Simulating perturbed retrieval")
sigma_pert = 1.5
r0 = exc.phi(patterns[0,0,:]) + sigma_pert*np.random.randn(exc.size)
net.simulate(T, r0=r0)

M_pert = np.mean(net.exc.state**2, axis=0)
overlaps_pert = sequences[0].overlaps(net, exc)
correlations_pert = sequences[0].overlaps(net, exc, correlation=True)
state_pert = net.exc.state

INFO:connectivity:Building connections from exc to exc
INFO:connectivity:Storing sequences
100%|██████████| 1/1 [00:04<00:00,  4.19s/it]58.02it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix
INFO:root:Simulating normal retrieval
INFO:network:Integrating network dynamics
[========================================================================] 100%
INFO:sequences:Computing overlaps
100%|██████████| 16/16 [00:00<00:00, 67.09it/s]
INFO:sequences:Computing correlations
100%|██████████| 16/16 [00:16<00:00,  1.02s/it]
INFO:root:Simulating perturbed retrieval
INFO:network:Integrating network dynamics
[========================================================================] 100%
INFO:sequences:Computing overlaps
100%|██████████| 16/16 [00:00<00:00, 63.24it/s]
INFO:sequences:Computing correlations
100%|██████████| 16/16 [00:38<00:00,  2.47s/it]


Save data

In [4]:
np.save(
    open("data/data_b.npy", "wb"),
    [M, overlaps, correlations, state.astype(np.float32),
     M_pert, overlaps_pert, correlations_pert, state_pert.astype(np.float32),
     phi_patterns.astype(np.float32)])